# Estudo da composição das previsões do rating final

O objetivo é realizar previsões para toda base gerando um rating unindo o contribuinte e posteriormente o igr gerado na regressão

In [2]:
import os
import dotenv
import boto3
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import date
import zipfile
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings

warnings.filterwarnings("ignore")

print("Processo iniciado")

# Define diretorios
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

Processo iniciado


In [ ]:
# print("Carregando dados para previsões")

# # Prepara os dados para realizar as previsões
# zip_file = os.path.join(dataPath, 'estoque_da3.zip') # 14bi
# z = zipfile.ZipFile(zip_file)


# def ler_bases_exportadas(nome_arquivo):
#     z.extract(nome_arquivo)
#     df = pd.read_csv(nome_arquivo, sep=',')
#     os.remove(nome_arquivo)
#     return df


# base_conjunta = ler_bases_exportadas('estoque-divida.csv')

In [3]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

def read_s3(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

In [4]:
print("Carregando dados para previsões")

dados_predict = read_s3(
     bucket_name=os.getenv("S3_BUCKET_NAME"), 
     folder_name=os.getenv("S3_FOLDER_NAME"), 
     file_name='feature_store_igr_prime_ab1_maxpg_lari.csv')

Carregando dados para previsões


In [5]:
dados_predict.to_csv(r'C:\Users\Consultor\Documents\bases_pesquisa\dados_predict.csv', index=False)

In [6]:
dados_contribuinte = read_s3(
    bucket_name = os.getenv("S3_BUCKET_NAME"), 
    folder_name = os.getenv("S3_FOLDER_NAME"), 
    file_name = 'feature_store_contribuinte_prime_k5_maxpg_luan.csv')

In [ ]:
dados_contribuinte.to_csv(r'C:\Users\Consultor\Documents\bases_pesquisa\dados_contribuinte.csv', index=False)

In [24]:
formatar_moeda(dados_predict['valor_aberto'].sum())

'R$ 13.863.282.107,90'

In [25]:
dados_predict.shape[0]

967079

In [26]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
}
dados_predict = dados_predict.rename(columns=colunas_nome)

In [27]:
dados_predict.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'protesto', 'idade_divida', 'da_aberto', 'valor_total_da', 'valor_pago',
       'valor_aberto', 'dif_tot_pago', 'dif_tot_pago_aberto',
       'percentual_pago_cda', 'percentual_pago_cda_faixas'],
      dtype='object')

In [28]:
dados_contribuinte.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'valor_tot', 'valor_pago', 'qtd_notas_2anos', 'edificacao', 'situacao',
       'cpf_cnpj_existe', 'endereco_existe', 'perfil_acessivel',
       'situacao_cobranca', 'historico_pagamento_em_valor',
       'class_situacao_cobranca', 'class_contribuinte', 'da_aberto',
       'class_contribuinte_nome', 'class_contribuinte_perfil',
       'percentual_pago_cda', 'class_contribuinte_peso'],
      dtype='object')

In [29]:
# Filtrando variáveis de interesse para a modelagem
df_divida = dados_predict[['cda', 
                          'tipo_divida',
                          'id_pessoa', 
                          'da_aberto',
                          'percentual_pago_cda', 
                          'valor_total_da', 
                          'idade_divida'
                          ]] 

In [30]:
dados_contribuinte_da_ab_1 = dados_contribuinte[dados_contribuinte['da_aberto'] == 1]

In [31]:
dados_contribuinte_da_ab_1.shape[0]

279212

In [32]:
df_contribuinte = dados_contribuinte_da_ab_1[['id_pessoa', 
                                      'tipo_divida',
                                      'num_dist_cda', 
                                      'quantidade_reparcelamento',
                                       'historico_pagamento_em_valor',
                                       'situacao_cobranca',
                                       'class_contribuinte',
                                       'class_contribuinte_peso']]

In [33]:
df = pd.merge(df_divida, df_contribuinte, on=['id_pessoa', 'tipo_divida'], how = "left")
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [34]:
# Métricas p acompanhar
num_cda_01 = df['cda'].nunique() # 967.079
num_pessoa_01 = df['id_pessoa'].nunique() # 255.253
vlr_da_01 = df['valor_total_da'].sum() # 'R$ 13.863.337.109,58'

In [35]:
num_cda_01

967079

In [36]:
num_pessoa_01

255253

In [37]:
formatar_moeda(vlr_da_01)

'R$ 13.863.337.109,58'

In [38]:
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [39]:
print("Preparação do pipeline de previsões")
# Filtra os dados que precisamos para previsão
df_feature_store = df[['valor_total_da', 
                       'idade_divida', #'anos_idade_da',
                        'quantidade_reparcelamento',
                       'num_dist_cda', #'frequencia_da_pessoa',
                        #'historico_pagamento_em_qtd', 
                        'situacao_cobranca', #'status_situacao',
                       'historico_pagamento_em_valor', 
                       'class_contribuinte_peso']]
df_feature_store.shape[0]


Preparação do pipeline de previsões


967079

In [40]:
# Normaliza os dados para previsão utilizando o StandardScaler
normalizador = StandardScaler() # RobustScaler()
normalizador.fit(df_feature_store)
dados_normalizados = normalizador.fit_transform(df_feature_store)

colunas = list(normalizador.get_feature_names_out())
df_normalizado = pd.DataFrame(dados_normalizados, columns=colunas)

In [41]:
df_normalizado.shape[0]

967079

In [42]:
print("Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)")


def abre_modelo(nome_modelo, path_modelo, zip_name=None):
    if zip_name:
        zip_file = os.path.join(path_modelo, zip_name)
        z = zipfile.ZipFile(zip_file)
        z.extract(nome_modelo)
    else:
        nome_modelo = os.path.join(path_modelo, nome_modelo)

    modelo = pickle.load(open(nome_modelo, 'rb'))
    return modelo


model_predict_igr = abre_modelo("modeloDA-igr-divida-prime_maxpg_k5_luan.pkl", modelsPath)

Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)


In [43]:
# Realizando previsões
previsoes = model_predict_igr.predict(df_normalizado)
df['igr'] = previsoes

In [44]:
df[df['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,5,0.53375,0.0


In [45]:
df.loc[df['situacao_cobranca'] == 0, 'igr'] = 0

In [46]:
df.loc[df['idade_divida'] >= 15, 'igr'] = 0

In [47]:
df.shape[0]

967079

In [48]:
df[df['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,5,0.53375,0.0


In [49]:
dados_contribuinte.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'valor_tot', 'valor_pago', 'qtd_notas_2anos', 'edificacao', 'situacao',
       'cpf_cnpj_existe', 'endereco_existe', 'perfil_acessivel',
       'situacao_cobranca', 'historico_pagamento_em_valor',
       'class_situacao_cobranca', 'class_contribuinte', 'da_aberto',
       'class_contribuinte_nome', 'class_contribuinte_perfil',
       'percentual_pago_cda', 'class_contribuinte_peso'],
      dtype='object')

In [27]:
# #dados_contribuinte.loc[dados_contribuinte['class_contribuinte'] == 4, ['class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_perfil']]
# contrib = dados_contribuinte.groupby(['class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_perfil'])['id_pessoa'].nunique().to_frame().reset_index()

# dict = {"class_contribuinte_nome": ['PRIMEIRA DIVIDA', 'DEVEDOR EXCEL', 
#                                     'DEVEDOR BOM', 'DEVEDOR RUIM', 'DEVEDOR PESSIMO'],
#         "ordem_contrib": [0, 1, 2 ,3, 4]}

# df_ord_class_contrib = pd.DataFrame(dict)

# contrib = pd.merge(contrib, df_ord_class_contrib, on = 'class_contribuinte_nome').sort_values(by = "ordem_contrib")
# total = contrib['id_pessoa'].sum()
# contrib['perc'] = np.round(contrib['id_pessoa']/total, 5)
# contrib

,class_contribuinte,class_contribuinte_nome,class_contribuinte_perfil,id_pessoa,ordem_contrib,perc
4,4,PRIMEIRA DIVIDA,NOVO EM DÍVIDA,161663,0,0.29868
2,2,DEVEDOR EXCEL,PAGA NAO REPARC,128380,1,0.23719
1,1,DEVEDOR BOM,PAGA REPARC,33664,2,0.06220
3,3,DEVEDOR RUIM,REPARCELADOR,3922,3,0.00725
0,0,DEVEDOR PESSIMO,NAO PAGA,213633,4,0.39469


# Rótulos da Dívida ALTERADOS!! 30/10

In [50]:
def make_rating_divida_original(dataframe):

    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Devedor Péssimo (NAO PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 2) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Devedor ruim (PAGA POUCO)
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.9), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.9) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.6) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Devedor Contumaz (FREQUENTE EM DÍVIDA)
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.9), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.9) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.6) & (dataframe['igr'] >= 0.2), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.2) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Devedor bom (PAGA REPARCELADO)
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.7), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.7) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.3) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Devedor Excelente (PAGA)
    # dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] >= 0.4), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] < 0.4) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    ## PRIMEIRA DIVIDA (NOVO EM DIVIDA)
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] < 0.3) & (dataframe['igr'] >= 0.1) , 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0) , 'rating_divida'] = 'MEDIA'

# dicionario_clusteres = {
#     'class_contribuinte': [0, 1, 2, 3, 4, 5],
#     'class_contribuinte_nome': ['DEVEDOR EXCELENTE',
#                                 'DEVEDOR CONTUMAZ',
#                                 'DEVEDOR PESSIMO',
#                                 'DEVEDOR RUIM',
#                                 'DEVEDOR BOM',
#                                 'PRIMEIRA DIVIDA'],
 
#     'class_contribuinte_perfil': ['NAO PAGA',
#                                 'PAGA REPARCELADO',
#                                 'PAGA',
#                                 'FREQUENTE EM DÍVIDA',
#                                 'PAGA POUCO',                                
#                                 'NOVO EM DIVIDA']
# }
 
# df_dicionario_clusteres = pd.DataFrame(dicionario_clusteres)
# df_dicionario_clusteres
    return dataframe


In [123]:
def make_rating_divida_k6(dataframe):

    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Devedor Péssimo (NAO PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'


    # DEVEDOR RUIM (PAGA REPARCELADO)
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] >= 0.9), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] < 0.9) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 5) & (dataframe['igr'] < 0.6) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # DEVEDOR REPARCELADOR (PAGA POUCO)
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.6) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.3) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # DEVEDOR NEGOCIADOR (FREQUENTE EM DÍVIDA)
    dataframe.loc[(dataframe['class_contribuinte'] == 2) & (dataframe['igr'] >= 0.8), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 2) & (dataframe['igr'] < 0.8) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 2) & (dataframe['igr'] < 0.6) & (dataframe['igr'] >= 0.2), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 2) & (dataframe['igr'] < 0.2) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # DEVEDOR CONTUMAZ (FREQUENTE EM DÍVIDA)
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.9), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.9) & (dataframe['igr'] >= 0.6), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.6) & (dataframe['igr'] >= 0.2), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.2) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # DEVEDOR EXCELENTE (PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.3) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    ## PRIMEIRA DIVIDA (NOVO EM DIVIDA)
    dataframe.loc[(dataframe['class_contribuinte'] == 6) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 6) & (dataframe['igr'] < 0.3) & (dataframe['igr'] >= 0.1) , 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 6) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0) , 'rating_divida'] = 'MEDIA'

# dicionario_clusteres = {
#     'class_contribuinte': [0, 1, 2, 3, 4, 5, 6],
#     'class_contribuinte_nome': ['DEVEDOR PESSIMO',
#                                 'DEVEDOR REPARCELADOR',
#                                 'DEVEDOR NEGOCIADOR',
#                                 'DEVEDOR CONTUMAZ',
#                                 'DEVEDOR EXCELENTE',
#                                 'DEVEDOR RUIM',                                
#                                 'PRIMEIRA DIVIDA'],
 
#     'class_contribuinte_perfil': ['NAO PAGA',
#                                 'PAGA REPARCELADO',
#                                 'PAGA NEGOCIANDO',
#                                 'FREQUENTE EM DÍVIDA',
#                                 'PAGA BEM', 
#                                 'PAGA POUCO',                               
#                                 'NOVO EM DIVIDA']
# }
 
# df_dicionario_clusteres = pd.DataFrame(dicionario_clusteres)
# df_dicionario_clusteres
    return dataframe


### Outra abordagem

In [33]:
def make_rating_divida_quantil(dataframe):
    
    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Devedor Excelente (PAGA NAO REPARC)
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'


    # Devedor bom (PAGA REPARC)
    q075_dev_bom = dataframe.loc[(dataframe['class_contribuinte'] == 1), 'igr'].quantile(0.75)
    q050_dev_bom = dataframe.loc[(dataframe['class_contribuinte'] == 1), 'igr'].quantile(0.5)

    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= q075_dev_bom), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < q075_dev_bom) & (dataframe['igr'] >= q050_dev_bom), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < q050_dev_bom) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'


    # Devedor ruim (REPARCELADOR)
    q075_dev_ruim = dataframe.loc[(dataframe['class_contribuinte'] == 3), 'igr'].quantile(0.75)
    q050_dev_ruim = dataframe.loc[(dataframe['class_contribuinte'] == 3), 'igr'].quantile(0.5)

    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= q075_dev_ruim), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < q075_dev_ruim) & (dataframe['igr'] >= q050_dev_ruim), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < q050_dev_ruim) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'


    # Devedor Péssimo (NAO PAGA)
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'


    ## PRIMEIRA DIVIDA (class_contribuinte == 4)
    dataframe.loc[(dataframe['class_contribuinte'] == 4), 'rating_divida'] = 'ALTA'


    return dataframe

In [51]:
print("Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método K5")
df_original = make_rating_divida_original(df)
df_original

Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método K5


,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.0,2048.73,1.0,2,0,0.0,4.0,0,1.66216,0.779436,ALTISSIMA
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.0,2515.85,1.0,3,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,1,0.0,847.02,0.0,4,3,0.0,3.0,0,1.66216,0.621546,ALTISSIMA
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,1,0.0,5385.81,11.0,4,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,1,0.0,6434.96,12.0,4,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,1,0.0,5082.97,11.0,5,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,1,0.0,7634.36,11.0,6,0,0.0,2.0,3,-1.02688,0.235812,BAIXA
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1,0.0,1080.38,1.0,3,2,0.0,3.0,0,1.66216,0.384772,ALTA
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,1,0.0,9390.43,25.0,5,0,0.0,4.0,0,1.66216,0.000000,BAIXISSIMA


In [52]:
df_original[df_original['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,5,0.53375,0.0,BAIXISSIMA


In [57]:
# print("Classificação do rating da dívida parametrizando junto a classificação do contribuinte - método QUANTIL")
# df_quantil = make_rating_divida_quantil(df)
# df_quantil

### Comparar versão original com quantil 

In [35]:
# df_orig_comparar = df_original[['cda', 'tipo_divida', 'id_pessoa', 'rating_divida']]
# df_orig_comparar = df_orig_comparar.rename(columns = {'rating_divida':'rating_divida_orig'})

In [36]:
# df_quantil_comparar = df_quantil[['cda', 'tipo_divida', 'id_pessoa', 'rating_divida']]
# df_quantil_comparar = df_quantil_comparar.rename(columns = {'rating_divida':'rating_divida_quantil'})

In [37]:
# comparar = pd.merge(df_orig_comparar, df_quantil_comparar, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'outer')

# rotulos_diferentes = comparar[comparar['rating_divida_orig'] != comparar['rating_divida_quantil']]

In [38]:
# comparar#[['rating_divida_orig', 'rating_divida_quantil']].value_counts().to_frame().reset_index().sort_values(by = ['rating_divida_orig', 'rating_divida_quantil'])

,cda,tipo_divida,id_pessoa,rating_divida_orig,rating_divida_quantil
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,ALTISSIMA,ALTISSIMA
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,BAIXISSIMA,BAIXISSIMA
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,BAIXISSIMA,BAIXISSIMA
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,BAIXISSIMA,BAIXISSIMA
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,BAIXISSIMA,BAIXISSIMA
...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,BAIXISSIMA,BAIXISSIMA
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,BAIXISSIMA,BAIXISSIMA
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,BAIXISSIMA,BAIXISSIMA
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,BAIXISSIMA,BAIXISSIMA


In [58]:
# contingency_table = pd.crosstab(comparar['rating_divida_orig'], comparar['rating_divida_quantil'])
# contingency_table

# Métricas

In [53]:
dict = {"rating_divida": ['ALTISSIMA', 'ALTA', 'MEDIA', 'BAIXA', 'BAIXISSIMA'],
        "ordem_rating": [0, 1, 2, 3, 4]}

df_ordenar_rating_divida = pd.DataFrame(dict)
df_ordenar_rating_divida

,rating_divida,ordem_rating
0,ALTISSIMA,0
1,ALTA,1
2,MEDIA,2
3,BAIXA,3
4,BAIXISSIMA,4


In [54]:
def numcdas_por_ratingdivida(DF, df_ordenar_rating_divida):

    # Num CDAs
    df_cdasabertas_ratdiv = DF.groupby('rating_divida')['cda'].nunique().to_frame().reset_index()
    total_cda = df_cdasabertas_ratdiv['cda'].sum()
    df_cdasabertas_ratdiv['total_cda'] = total_cda
    df_cdasabertas_ratdiv['perc_cda'] = np.round(df_cdasabertas_ratdiv['cda']/total_cda, 4)

    df_cdasabertas_ratdiv = pd.merge(df_cdasabertas_ratdiv, df_ordenar_rating_divida, on = 'rating_divida').sort_values(by = "ordem_rating")

    return df_cdasabertas_ratdiv

In [55]:
def vlrmonet_por_ratingdivida(DF, df_ordenar_rating_divida):

    # Valor monetário
    estoque_rat_div = DF.groupby('rating_divida')['valor_total_da'].sum().to_frame().reset_index()
    total_vlr_tot_da = estoque_rat_div['valor_total_da'].sum()
    estoque_rat_div['total_vlr_tot_da'] = total_vlr_tot_da
    estoque_rat_div['perc_vlr_tot_da'] = np.round(estoque_rat_div['valor_total_da']/total_vlr_tot_da, 4)

    estoque_rat_div['valor_total_da_format'] = estoque_rat_div['valor_total_da'].apply(formatar_moeda, 0)
    estoque_rat_div['total_vlr_tot_da_format'] = estoque_rat_div['total_vlr_tot_da'].apply(formatar_moeda, 0)

    estoque_rat_div = pd.merge(estoque_rat_div, df_ordenar_rating_divida, on = 'rating_divida').sort_values(by = "ordem_rating")

    return estoque_rat_div

### Original

In [56]:
df_orig_da1 = df_original[df_original['da_aberto'] == 1]

In [57]:
numcdas_por_ratingdivida(df_orig_da1, df_ordenar_rating_divida)

,rating_divida,cda,total_cda,perc_cda,ordem_rating
1,ALTISSIMA,207482,967079,0.2145,0
0,ALTA,16925,967079,0.0175,1
4,MEDIA,115773,967079,0.1197,2
2,BAIXA,170296,967079,0.1761,3
3,BAIXISSIMA,456603,967079,0.4721,4


In [58]:
vlrmonet_por_ratingdivida(df_orig_da1, df_ordenar_rating_divida)

,rating_divida,valor_total_da,total_vlr_tot_da,perc_vlr_tot_da,valor_total_da_format,total_vlr_tot_da_format,ordem_rating
1,ALTISSIMA,1.077471e+09,1.386334e+10,0.0777,"R$ 1.077.471.347,87","R$ 13.863.337.109,58",0
0,ALTA,2.181371e+08,1.386334e+10,0.0157,"R$ 218.137.059,56","R$ 13.863.337.109,58",1
4,MEDIA,1.563912e+09,1.386334e+10,0.1128,"R$ 1.563.912.185,95","R$ 13.863.337.109,58",2
2,BAIXA,2.118692e+09,1.386334e+10,0.1528,"R$ 2.118.691.764,08","R$ 13.863.337.109,58",3
3,BAIXISSIMA,8.885125e+09,1.386334e+10,0.6409,"R$ 8.885.124.752,12","R$ 13.863.337.109,58",4


### Quantil

In [46]:
# df_quant_da1 = df_quantil[df_quantil['da_aberto'] == 1]

In [65]:
# numcdas_por_ratingdivida(df_quant_da1, df_ordenar_rating_divida)

In [66]:
# vlrmonet_por_ratingdivida(df_quant_da1, df_ordenar_rating_divida)

### Métricas

In [59]:
metricas_predict = df.groupby('rating_divida')['cda'].nunique().to_frame().reset_index()
total = metricas_predict['cda'].sum()
metricas_predict['perc'] = np.round(metricas_predict['cda']/total, 5)
metricas_predict

,rating_divida,cda,perc
0,ALTA,16925,0.01750
1,ALTISSIMA,207482,0.21455
2,BAIXA,170296,0.17609
3,BAIXISSIMA,456603,0.47215
4,MEDIA,115773,0.11971


In [60]:
df_orig_da1[df_orig_da1['cda'] == '1491939ceb02c7cfe52281a70553db462']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77158,1491939ceb02c7cfe52281a70553db462,mercantil,2f48e5117fc31832,1,0.0,3298.56,1.0,1,0,0.0,0.0,5,0.53375,0.0,BAIXISSIMA


In [61]:
df_orig_da1.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso',
       'igr', 'rating_divida'],
      dtype='object')

In [62]:
df_original[df_original['cda'] == '14a8fa60925e446f9fcacff532cf50c42']

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
77522,14a8fa60925e446f9fcacff532cf50c42,mercantil,a6b1c07dd9a81e87,1,0.0,3298.56,1.0,1,0,0.0,0.0,5,0.53375,0.0,BAIXISSIMA


In [63]:
df_original

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.0,2048.73,1.0,2,0,0.0,4.0,0,1.66216,0.779436,ALTISSIMA
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.0,2515.85,1.0,3,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
2,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,1,0.0,847.02,0.0,4,3,0.0,3.0,0,1.66216,0.621546,ALTISSIMA
3,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,1,0.0,5385.81,11.0,4,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
4,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,1,0.0,6434.96,12.0,4,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967074,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,1,0.0,5082.97,11.0,5,0,0.0,0.0,2,-1.20205,0.000000,BAIXISSIMA
967075,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,1,0.0,7634.36,11.0,6,0,0.0,2.0,3,-1.02688,0.235812,BAIXA
967076,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1,0.0,1080.38,1.0,3,2,0.0,3.0,0,1.66216,0.384772,ALTA
967077,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,1,0.0,9390.43,25.0,5,0,0.0,4.0,0,1.66216,0.000000,BAIXISSIMA


In [53]:
df_teste2 = pd.merge(df_original, 
                     dados_contribuinte, on=['id_pessoa', 'tipo_divida'], how = "left")

df_teste2

,cda,tipo_divida,id_pessoa,da_aberto_x,percentual_pago_cda_x,valor_total_da,idade_divida,num_dist_cda_x,quantidade_reparcelamento_x,historico_pagamento_em_valor_x,...,perfil_acessivel,situacao_cobranca_y,historico_pagamento_em_valor_y,class_situacao_cobranca,class_contribuinte_y,da_aberto_y,class_contribuinte_nome,class_contribuinte_perfil,percentual_pago_cda_y,class_contribuinte_peso_y
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.0,2048.73,1.0,2,0,0.0,...,2.0,4.0,0.000000,MUITO ACESSÍVEL,2,0,DEVEDOR EXCEL,PAGA,0.000000,1.64003
1,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.0,2048.73,1.0,2,0,0.0,...,2.0,4.0,0.000000,MUITO ACESSÍVEL,2,1,DEVEDOR EXCEL,PAGA,0.000000,1.64003
2,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.0,2515.85,1.0,3,0,0.0,...,0.0,0.0,0.000000,INACESSÍVEL,0,0,DEVEDOR PESSIMO,NAO PAGA,0.000000,-1.21376
3,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.0,2515.85,1.0,3,0,0.0,...,0.0,0.0,0.000000,INACESSÍVEL,0,1,DEVEDOR PESSIMO,NAO PAGA,0.000000,-1.21376
4,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,1,0.0,847.02,0.0,4,3,0.0,...,2.0,0.0,0.295438,INACESSÍVEL,1,0,DEVEDOR BOM,PAGA REPARCELADO,0.295438,-0.92367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837276,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,1,0.0,1080.38,1.0,3,2,0.0,...,2.0,0.0,0.000000,INACESSÍVEL,0,1,DEVEDOR PESSIMO,NAO PAGA,0.000000,-1.21376
1837277,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,1,0.0,9390.43,25.0,5,0,0.0,...,2.0,4.0,0.390289,MUITO ACESSÍVEL,1,0,DEVEDOR BOM,PAGA REPARCELADO,0.390289,-0.92367
1837278,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,1,0.0,9390.43,25.0,5,0,0.0,...,2.0,4.0,0.000000,MUITO ACESSÍVEL,2,1,DEVEDOR EXCEL,PAGA,0.000000,1.64003
1837279,fffff192f894983c9ceed47545e621dd1,imovel,dd5538d2d38803a7,1,0.0,16594.80,15.0,3,0,0.0,...,2.0,4.0,0.073106,MUITO ACESSÍVEL,1,0,DEVEDOR BOM,PAGA REPARCELADO,0.073106,-0.92367


In [64]:
print("Inicia a conexão com S3 para inscrição dos dados com as previsões")

# Cria conexão ao s3 e preenche a tabela com os dados

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())


up_s3_files(dataframe = df_original,
            bucket_name = os.getenv("S3_BUCKET_NAME"),
            folder_name = os.getenv("S3_FOLDER_NAME"),  
            file_name = 'prime_maxpg_k5_luan.csv')

print("Upload de dados efetuados no s3")
print("Processo finalizado")
print("Arquivo disponível para download e análise")

Inicia a conexão com S3 para inscrição dos dados com as previsões
Upload de dados efetuados no s3
Processo finalizado
Arquivo disponível para download e análise
